In [1]:
import pandas as pd

# Load the dataset
file_path = 'family_financial_and_transactions_data.xlsx'  # Update with your file path
df = pd.read_excel(file_path, sheet_name='final_family_financial_and_tran')

# Display the first few rows
df.head()


,Family ID,Member ID,Transaction Date,Category,Amount,Income,Savings,Monthly Expenses,Loan Payments,Credit Card Spending,Dependents,Financial Goals Met (%)
0,FAM001,FAM001_Member1,2024-10-07,Travel,409.12,113810,20234,5781,2422,2959,2,68
1,FAM001,FAM001_Member1,2024-10-16,Travel,270.91,113810,20234,5781,2422,2959,2,68
2,FAM001,FAM001_Member1,2024-10-17,Groceries,91.10,113810,20234,5781,2422,2959,2,68
3,FAM001,FAM001_Member1,2024-10-25,Healthcare,198.23,113810,20234,5781,2422,2959,2,68
4,FAM001,FAM001_Member1,2024-10-25,Education,206.42,113810,20234,5781,2422,2959,2,68


In [ ]:
# Check the structure of the data
df.info()

# Check for missing values
print("Missing values in each column:\n", df.isnull().sum())

# Check for duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())


In [ ]:
# Remove duplicates if they exist
df = df.drop_duplicates()
print("Data after removing duplicates:", df.shape)


In [ ]:
# Aggregate data at the family level
family_summary = df.groupby('Family ID').agg(
    Total_Income=('Income', 'mean'),
    Total_Savings=('Savings', 'mean'),
    Total_Monthly_Expenses=('Monthly Expenses', 'mean'),
    Total_Loan_Payments=('Loan Payments', 'mean'),
    Total_Credit_Spending=('Credit Card Spending', 'mean'),
    Average_Financial_Goals_Met=('Financial Goals Met (%)', 'mean'),
    Total_Transaction_Amount=('Amount', 'sum')
).reset_index()

# Display family-level summary
family_summary.head()


In [ ]:
# Aggregate data at the member level
member_summary = df.groupby(['Family ID', 'Member ID']).agg(
    Total_Income=('Income', 'mean'),
    Total_Savings=('Savings', 'mean'),
    Total_Monthly_Expenses=('Monthly Expenses', 'mean'),
    Total_Loan_Payments=('Loan Payments', 'mean'),
    Total_Credit_Spending=('Credit Card Spending', 'mean'),
    Average_Financial_Goals_Met=('Financial Goals Met (%)', 'mean'),
    Total_Transaction_Amount=('Amount', 'sum')
).reset_index()

# Display member-level summary
member_summary.head()


In [ ]:
# Calculate total spending by category
category_spending = df.groupby('Category')['Amount'].sum().reset_index()

# Add a percentage column for each category
category_spending['Percentage (%)'] = (category_spending['Amount'] / category_spending['Amount'].sum()) * 100

# Display the spending by category
category_spending.sort_values(by='Percentage (%)', ascending=False)


In [ ]:
import matplotlib.pyplot as plt

# Create a bar chart for spending by category
plt.figure(figsize=(10, 6))
plt.bar(category_spending['Category'], category_spending['Amount'], alpha=0.8)
plt.xticks(rotation=45, ha='right')
plt.title('Total Spending by Category')
plt.xlabel('Category')
plt.ylabel('Amount')
plt.show()


In [ ]:
# Calculate total spending by family and category
family_spending = df.groupby(['Family ID', 'Category'])['Amount'].sum().reset_index()

# Normalize spending to calculate percentage within each family
family_spending['Percentage (%)'] = family_spending.groupby('Family ID')['Amount'].transform(lambda x: (x / x.sum()) * 100)

# Display family-wise spending patterns
family_spending.head()


In [ ]:
import seaborn as sns

# Create a heatmap to visualize family-wise spending
family_pivot = family_spending.pivot(index='Family ID', columns='Category', values='Percentage (%)')

plt.figure(figsize=(12, 8))
sns.heatmap(family_pivot, cmap='YlGnBu', annot=True, fmt=".1f")
plt.title('Family-Wise Spending Distribution (%)')
plt.show()


In [ ]:
# Select relevant financial columns for correlation
financial_metrics = family_summary[['Total_Income', 'Total_Savings', 'Total_Monthly_Expenses', 
                                     'Total_Loan_Payments', 'Total_Credit_Spending', 
                                     'Average_Financial_Goals_Met']]

# Compute the correlation matrix
correlation_matrix = financial_metrics.corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Between Financial Metrics', fontsize=14)
plt.show()


In [ ]:
# Check if Financial_Score exists in the DataFrame
print(family_data_normalized.columns)

# Plot histogram if the Financial_Score column exists
if 'Financial_Score' in family_data_normalized.columns:
    plt.figure(figsize=(8, 6))
    plt.hist(family_data_normalized['Financial_Score'], bins=10, color='skyblue', edgecolor='black')
    plt.title('Distribution of Financial Scores', fontsize=16)
    plt.xlabel('Financial_Score', fontsize=12)
    plt.ylabel('Number of Families', fontsize=12)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()
else:
    print("Financial_Score column not found in the DataFrame.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame with all the data loaded and preprocessed

# Step 1: Calculate Ratios for Each Family
family_data = df.groupby('Family ID').agg({
    'Income': 'mean',
    'Savings': 'mean',
    'Monthly Expenses': 'mean',
    'Loan Payments': 'mean',
    'Credit Card Spending': 'mean',
    'Financial Goals Met (%)': 'mean'
}).reset_index()

family_data['Savings_to_Income_Ratio'] = family_data['Savings'] / family_data['Income']
family_data['Expenses_to_Income_Percentage'] = family_data['Monthly Expenses'] / family_data['Income']
family_data['Loan_to_Income_Percentage'] = family_data['Loan Payments'] / family_data['Income']

# Step 2: Analyze Credit Card Spending Trends (use the average Credit Card Spending)
# For simplicity, we assume this is already handled in the aggregation

# Step 3: Calculate Spending Category Distribution
discretionary_categories = ['Travel', 'Entertainment', 'Food']

# Total spending per family
total_spending = df.groupby('Family ID')['Amount'].sum().reset_index()
total_spending.rename(columns={'Amount': 'Total_Spending'}, inplace=True)

# Spending on discretionary categories per family
discretionary_spending = df[df['Category'].isin(discretionary_categories)].groupby('Family ID')['Amount'].sum().reset_index()
discretionary_spending.rename(columns={'Amount': 'Discretionary_Spending'}, inplace=True)

# Merge total spending and discretionary spending
spending_data = pd.merge(total_spending, discretionary_spending, on='Family ID', how='left')
spending_data['Discretionary_Spending'].fillna(0, inplace=True)

# Calculate Discretionary Spending Percentage
spending_data['Discretionary_Spending_Percentage'] = spending_data['Discretionary_Spending'] / spending_data['Total_Spending']

# Merge spending data with family data
family_data = pd.merge(family_data, spending_data[['Family ID', 'Discretionary_Spending_Percentage']], on='Family ID', how='left')

# Step 4: Normalize the Factors
scaler = MinMaxScaler()

# Factors to normalize
factors_to_normalize = [
    'Savings_to_Income_Ratio',
    'Expenses_to_Income_Percentage',
    'Loan_to_Income_Percentage',
    'Credit Card Spending',
    'Discretionary_Spending_Percentage',
    'Financial Goals Met (%)'
]

# Apply Min-Max Scaling
family_data_normalized = family_data.copy()
family_data_normalized[factors_to_normalize] = scaler.fit_transform(family_data[factors_to_normalize])

# Step 5: Calculate the Financial Score
weights = {
    'Savings_to_Income_Ratio': 0.25,
    'Expenses_to_Income_Percentage': 0.20,
    'Loan_to_Income_Percentage': 0.15,
    'Credit Card Spending': 0.10,
    'Discretionary_Spending_Percentage': 0.20,
    'Financial Goals Met (%)': 0.10
}

# Calculate the weighted score
family_data_normalized['Financial_Score'] = (
    family_data_normalized['Savings_to_Income_Ratio'] * weights['Savings_to_Income_Ratio'] +
    (1 - family_data_normalized['Expenses_to_Income_Percentage']) * weights['Expenses_to_Income_Percentage'] +
    (1 - family_data_normalized['Loan_to_Income_Percentage']) * weights['Loan_to_Income_Percentage'] +
    (1 - family_data_normalized['Credit Card Spending']) * weights['Credit Card Spending'] +
    (1 - family_data_normalized['Discretionary_Spending_Percentage']) * weights['Discretionary_Spending_Percentage'] +
    family_data_normalized['Financial Goals Met (%)'] * weights['Financial Goals Met (%)']
)

# Scale the score to 0-100
family_data_normalized['Financial_Score'] = family_data_normalized['Financial_Score'] * 100

# Define score categories
def score_category(score):
    if score < 50:
        return 'Low'
    elif 50 <= score < 75:
        return 'Average'
    else:
        return 'Good'

# Apply score categories to the data
family_data_normalized['Score_Category'] = family_data_normalized['Financial_Score'].apply(score_category)

# Step 6: Generate Recommendations
def generate_recommendations(row):
    recommendations = []
    
    # Low score recommendations
    if row['Score_Category'] == 'Low':
        if row['Savings_to_Income_Ratio'] < 0.2:
            recommendations.append("Increase savings to at least 20% of your income.")
        if row['Expenses_to_Income_Percentage'] > 0.5:
            recommendations.append("Reduce monthly expenses to below 50% of your income.")
        if row['Loan_to_Income_Percentage'] > 0.2:
            recommendations.append("Aim to reduce loan payments to below 20% of your income.")
        if row['Credit Card Spending'] > 0.3:
            recommendations.append("Reduce credit card spending to below 30% of total spending.")
        if row['Financial Goals Met (%)'] < 70:
            recommendations.append("Focus on achieving financial goals to at least 70%.")
    
    # Average score recommendations
    elif row['Score_Category'] == 'Average':
        if row['Savings_to_Income_Ratio'] < 0.3:
            recommendations.append("Consider increasing savings to 30% of your income.")
        if row['Expenses_to_Income_Percentage'] > 0.4:
            recommendations.append("Try to reduce monthly expenses further.")
    
    # Good score recommendations
    else:
        recommendations.append("You're on track! Keep maintaining a healthy balance between savings and expenses.")
    
    return recommendations

# Add recommendations to the DataFrame
family_data_normalized['Recommendations'] = family_data_normalized.apply(generate_recommendations, axis=1)

# Step 7: Merge financial scores back into the original dataset
df_with_scores = pd.merge(df, family_data_normalized[['Family ID', 'Financial_Score', 'Score_Category', 'Recommendations']], on='Family ID', how='left')

# Save the updated dataset to a CSV file
output_full_data_path = "complete_family_financial_data_with_scores_and_recommendations.csv"
df_with_scores.to_csv(output_full_data_path, index=False)

# Display the final scores and recommendations for families
final_scores_with_recommendations = family_data_normalized[['Family ID', 'Financial_Score', 'Score_Category', 'Recommendations']]
print(final_scores_with_recommendations)

print(f"Complete financial data with scores and recommendations saved to: {output_full_data_path}")
